In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

# Advanced D3
<!-- requirement: small_data/Charm_City_Circulator_Ridership.csv -->
<!-- requirement: projects/babyname -->
<!-- requirement: small_data/Charm_City_Circulator_Headways.csv -->

In this notebook, we use D3 to:
* build an interactive chart, purely with D3
* integrate Angular.js with D3 to build an interactive Bar chart
* show how to live querying a backend using D3 and AJAX calls

This is actually a notebook on pure D3. Packages like NVD3 allow for a quicker and smoother introduction to D3, which is presented elsewhere. Writing applications solely with D3 is more involved. In this notebook, we systematically proceed through a full D3 example.

We will build a simple scatterplot, and then customize it by coloring the dots, adding axes and legends, etc. We will also showcase some interactivity: **people love being able to click on things!**

Last, this notebook also includes a discussion and code on live-quering a backend with D3 and AJAX.

### Dataset and plan
One of the collaborators of this notebook loves Baltimore (his hometown) and public transit, so we'll be combining these two interests via [Baltimore's open datasets](https://data.baltimorecity.gov). In particular, we'll look at the [Charm City Circulator](https://en.wikipedia.org/wiki/Charm_City_Circulator), a free bus initiative for downtown Baltimore launched in 2010. 

- First, we'll take a look at the headways data, and after some basic aggregation we'll take a look at a plot of headways data vs. ridership numbers on a per-day basis.
- Next, we will use [D3](https://d3js.org/) to look at some basic vital signs. How many people ride daily? How full is each bus? For a very rough comparison, Maryland's MTA averages ~400,000 riders/weekday as of 2012.
- D3 is incredibly flexible but too primitive.  We'll use [NVD3](https://github.com/novus/nvd3) to simplify our plotting.
- We'll move on to breaking down ridership by weekday vs. weekend and line (unfortunately the released data does not get much more granular than this). Maybe we'll be able to spot useful trends, such as who is riding the bus (e.g. commuters vs. tourists - downtown buses like these could reasonably service both).

In [ ]:
# do some data science
import pandas as pd

# data described/found here
# https://data.baltimorecity.gov/browse?category=Transportation&limitTo=datasets&utf8=%E2%9C%93
with open("small_data/Charm_City_Circulator_Headways.csv") as fh:
    headways = pd.read_csv(fh)
    headways['date'] = pd.to_datetime(headways['date'])

with open("small_data/Charm_City_Circulator_Ridership.csv") as fh:
    ridership = pd.read_csv(fh)
    ridership['date'] = pd.to_datetime(ridership['date'])

ridership_averages_by_weekday = (
    ridership[
        ['day', 'orangeAverage', 'purpleAverage', 'greenAverage']
    ].groupby('day', sort=True).mean()) # get excited for lots of chaining in D3

ridership_averages_by_year = ridership[['orangeAverage', 'purpleAverage', 'greenAverage']]

In [ ]:
ridership_averages_by_year.loc[:,'year'] = ridership['date'].dt.year

In [ ]:
ridership_averages_by_year = ridership_averages_by_year.groupby('year')\
                                                       .mean()
def timeToFloat(timeString):
    try:
        minutes, seconds = timeString.split(":")
    except:
        return None
    return float(minutes) + float(seconds)/60.

In [ ]:
headways_averages = headways[['day', 'date', 'orangeHeadway', 'purpleHeadway', 'greenHeadway']]
ridership_averages = ridership[['day', 'date', 'orangeAverage', 'purpleAverage', 'greenAverage']]
merged = headways_averages.merge(ridership_averages)
merged['purpleHeadway'] = merged['purpleHeadway'].apply(timeToFloat)
merged_json = merged.to_json()
ridership_averages_by_year

## A Straightforward Scatterplot in D3
Cool, so we have some basic things to plot. Let's do some D3 now.

First, let's convert the data from pandas' strange JSON format into an array of objects, 1 per data point, with the expected attributes (what D3 [expects](https://github.com/mbostock/d3/wiki/Selections#data)):

In [ ]:
# import ihtml.py file for %%ihtml and %%jsdoc cell magic
import ihtml

In [ ]:
%%jsdoc plotData

window.headwayVsRidership = {{ merged | json }};

/*
 * As it turns out, pandas returns a dictionary with 
 * the top-level keys as the column headers, and values
 * as the list of values under that column.
 */
window.convertPlotData = function(data){
  var convertedData = [];
  // pandas gives each column the same number of
  // objects, arbitrarily choose one to iterate over
  for(var i in data[Object.keys(data)[0]]){
    var convertedDatum = {};
    for(var key in data){
      convertedDatum[key] = data[key][i];
    }
    convertedData.push(convertedDatum);
  }
  return convertedData;
}

Next, let's build a basic D3 scatter plot skeleton.

On the x-axis we'll plot the average headway that day, and on the y-axis we'll plot the average number of riders that day.

D3 works on 'method-chaining', much like pandas or jQuery, so each function call returns another object that we can operate on. For example, in the code below, when we call `d3.select(#chart1)`, the `select` function returns a D3-ified version of the JS object representing the `chart1` div we appended.

What makes D3 unique is the idea that _data_ is bound to _DOM Elements_ in a data binding step that we'll see below. This elegant abstraction means we can bind our data to any HTML elements we want and visualize them basically however we want to.

For our visualization, we'll use [SVG](https://developer.mozilla.org/en-US/docs/Web/SVG), which allows for relatively easy drawing of elements on a 2-dimensional canvas.

In [ ]:
%%jsdoc chart1

window.Chart = function(d3, data, name) {
  var selector = "#" + name;
  this.data = data;

  // the first thing we want to do is set up our viewport
  var containerWidth = 764;   // 780 approx width - 16 body margin
  var containerHeight = 504;  // 520 height - 16 body margin
  $(selector).width(containerWidth + "px");
  $(selector).height(containerHeight + "px");

  var margin = {top: 20, right: 20, bottom: 20, left: 60};
  this.width = containerWidth - margin.left - margin.right;
  this.height = containerHeight - margin.top - margin.bottom;
  this.svg = d3.select(selector).append("svg")
    .style("position", "relative")
    .attr("width", containerWidth + "px")
    .attr("height", containerHeight + "px")
    .append("g") // D3 returns us this 'g' element, it will be our main canvas
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")"); // axes will go in margins
    
  // D3 likes passing functions around, making these useful
  this.xVals = function(d){return d.purpleHeadway;};
  this.yVals = function(d){return d.purpleAverage;};

  // set the min/max values of our axes
  // the 1 is for a bit of padding
  this.xScale = d3.scale.linear()
    .range([0, this.width])
    .domain([d3.min(this.data, this.xVals)-1, d3.max(this.data, this.xVals)+1]);
  this.xMap = _.compose(this.xScale, this.xVals);

  // same thing for y axis
  this.yScale = d3.scale.linear()
    .range([this.height, 0])
    .domain([d3.min(this.data, this.yVals)-1, d3.max(this.data, this.yVals)+1]);
  this.yMap = _.compose(this.yScale, this.yVals);
};

In [ ]:
%%ihtml 520
<html>
    <head>
        <script src="https://code.jquery.com/jquery-3.1.0.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.6/d3.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/underscore.js/1.8.3/underscore-min.js"></script>
        {{ plotData | jsdoc }}
        {{ chart1 | jsdoc }}
        <script>
$(document).ready(function() {
  var data = window.convertPlotData(window.headwayVsRidership);
  var chart = new window.Chart(d3, data, "chart1");

  // this is the core 'D3' step
  chart.svg.selectAll(".dot") // note this begins as empty selection
    .data(chart.data) // data-binding step! this selection is now bound to the elements in our data array
    .enter() // new node for each datum not in selection
    .append("circle") // 1 for each new node
    .attr("class", "dot")
    .attr("r", 3.5) // radius
    .attr("cx", chart.xMap) // x position
    .attr("cy", chart.yMap) // y position: position 0 is top, (500 - margin) is bottom
});
        </script>
    </head>
    <body>
        <div id="chart1"></div>
    </body>
</html>

### So, what just happened?
Let's dive a little farther down the rabbit hole. 

If you examine the code, it becomes apparent that 

```javascript
 svg.selectAll(el)
   .data(data)
```

was the snippet of code that actually created the visualization. D3 works on a *data-binding* model, which means that you attach the data to an element (in particular `el` is an SVG window in this case), and D3 works its magic by creating an HTML object for every element in that dataset. 

- Step 1 in the binding is to select the HTML elements that will represent our data points
- Step 2 is to actually bind the data 
- Step 3, the functions after `data` is called, actually create, style, and position the elements that D3 will create based on the newly-bound data

## Customization: Axes

Now, we want to prettify this basic plot with axes, make it a bit more interactive, and so on. Moreover, D3 has several convenience functions we can use so we don't have to, for example, make/scale our own axes or data points on the plot.

In [ ]:
%%jsdoc chart2

window.ChartAxes = function(d3, data, name) {
  window.Chart.call(this, d3, data, name);
    
  // d3 also has an axis convenience function, and 
  // it even lets us pass our scale in to generate ticks
    
  this.xAxis = d3.svg.axis().scale(this.xScale).orient("bottom");
  this.yAxis = d3.svg.axis().scale(this.yScale).orient("left");
    
  // x-axis
  this.svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + this.height + ")") // slide it to bottom
      .call(this.xAxis) // this is a d3 idiom for SVG generation
      .append("text") // and we want to set a label
      .attr("class", "label")
      .attr("x", this.width)
      .attr("y", -15)
      .style("text-anchor", "end")
      .text("Average Headway");

  // y-axis
  this.svg.append("g")
      .attr("class", "y axis")
      .call(this.yAxis)
      .append("text")
      .attr("class", "label")
      .attr("transform", "rotate(-90)") // so text is vertical
      .attr("y", 0)
      .attr("dy", "1em")
      .style("text-anchor", "end")
      .text("Average Riders");
}

In [ ]:
%%ihtml 520
<html>
    <head>
        <script src="https://code.jquery.com/jquery-3.1.0.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.6/d3.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/underscore.js/1.8.3/underscore-min.js"></script>
        {{ plotData | jsdoc }}
        {{ chart1 | jsdoc }}
        {{ chart2 | jsdoc }}
        <script>
$(document).ready(function() {
  var data = window.convertPlotData(window.headwayVsRidership);
  var chart = new window.ChartAxes(d3, data, "chart2");

  chart.svg.selectAll(".dot")
    .data(chart.data)
    .enter()
    .append("circle") 
    .attr("class", "dot")
    .attr("r", 3.5)
    .attr("cx", chart.xMap) 
    .attr("cy", chart.yMap);
});
        </script>
    </head>
    <body>
        <div id="chart2"></div>
    </body>
</html>

## Customization: Coloring Datapoints

So this is a very basic, passable graph. However, it's not much better than something we could do in matplotlib. Let's add some interactivity, labeling, and color the dots to make differentiating weekdays/weekends easy.

In [ ]:
%%jsdoc chart3

window.ChartAxesColor = function(d3, data, name) {
  window.ChartAxes.call(this, d3, data, name);
    
  // NEW: Coloring. We now color the graph, based on 
  // whether it's a weekend or weekday.
  // As with our other scaling, we determine the color
  // on a per-datum basis and let D3 do much of the heavy lifting.
  this.cValue = function(d) {
    return (d.day == "Saturday" || d.day == "Sunday") ? "Weekend" : "Weekday";
  };
  this.color = d3.scale.category10(); //we'll now call this during our graph construction
    
  // NEW: TOOLTIP. 
  this.tooltip = d3.select("body").append("div")
    .attr("class", "tooltip")
    .style("opacity", 0)
    .style("background-color", "white");
}

window.ChartAxesColor.prototype.addLegend = function() {
  // NEW: Legend
  // Adding a legend has to be a function because it has to happen 
  // after the values of `this.color.domain()` are set
  var legend = this.svg.selectAll(".legend")
      .data(this.color.domain()) //stores the color <-> label mappings
      .enter().append("g")
      .attr("class", "legend")
      .attr("transform", function(d, i) { return "translate(0," + i * 20 + ")"; });

  // draw legend colored rectangles
  legend.append("rect")
      .attr("x", this.width - 60)
      .attr("width", 18)
      .attr("height", 18)
      .style("fill", this.color);

  // draw legend text
  legend.append("text")
      .attr("x", this.width - 70)
      .attr("y", 9)
      .attr("dy", ".35em")
      .style("text-anchor", "end")
      .text(function(d) { return d;})
}

In [ ]:
%%ihtml 520
<html>
    <head>
        <script src="https://code.jquery.com/jquery-3.1.0.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.6/d3.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/underscore.js/1.8.3/underscore-min.js"></script>
        {{ plotData | jsdoc }}
        {{ chart1 | jsdoc }}
        {{ chart2 | jsdoc }}
        {{ chart3 | jsdoc }}
        <script>
$(document).ready(function() {
  var data = window.convertPlotData(window.headwayVsRidership);
  var chart = new window.ChartAxesColor(d3, data, "chart3");

  chart.svg.selectAll(".dot")
     .data(chart.data)
     .enter().append("circle")
     .attr("class", "dot")
     .attr("r", 3.5)
     .attr("cx", chart.xMap)
     .attr("cy", chart.yMap)
     .style("fill", function(d) { return chart.color(chart.cValue(d));}) //D3 does the magic! 
     .on("mouseover", function(d) { //much like jquery, an event listener
         chart.tooltip.transition()
                .duration(200)
                .style("opacity", .9);
         chart.tooltip.html(d["day"] + " : " + d['date'])
               .style("left", (d3.event.pageX + 5) + "px")
               .style("top", (d3.event.pageY - 28) + "px");
      })
      .on("mouseout", function(d) {
          chart.tooltip.transition()
               .duration(500)
               .style("opacity", 0);
      });

  chart.addLegend();
});
        </script>
        <style>
            .tooltip {
                position: absolute;
            }
        </style>
    </head>
    <body>
        <div id="chart3"></div>
    </body>
</html>

## Easier D3 with NVD3

As it turns out, D3, while very extensible, takes a lot of work to use on its own, and we present it in a separate notebook. As good engineers, we're all about [looking for external libraries](https://www.google.com/webhp?&q=d3.js%20charts) when we want to do things better and more quickly. [NVD3](https://github.com/novus/nvd3) looks like a good candidate.

In [ ]:
%%jsdoc nvd3

window.averageByYear = {{ ridership_averages_by_year | json }}
window.averageByWeekday = {{ ridership_averages_by_weekday | json }}

// again this function was created just by inspecting
// the data format in the example, no substantive knowledge of D3 required
window.convertKeys = function(pandasJSON, keyOrder){
  var data = [];
  Object.keys(pandasJSON).forEach(function(key){
    var retObj = {};
    retObj['key'] = key;
    retObj['values'] = [];
    var color = key.substring(0, key.indexOf("A"));
    retObj['color'] = color;
    var order = Object.keys(pandasJSON[key]);
    if(keyOrder){
      order = keyOrder;
    }
    order.forEach(function(j){
      var innerObj = {};
      innerObj['x'] = j;
      innerObj['y'] = pandasJSON[key][j];
      retObj['values'].push(innerObj);
    });
    data.push(retObj);
  });
  return data;
};

window.WEEKDAYS_IN_ORDER = [
  "Monday",
  "Tuesday",
  "Wednesday",
  "Thursday",
  "Friday",
  "Saturday",
  "Sunday"
]

window.genGraph = function(data, el){
  return function() {
    var chart = nv.models.multiBarChart()
      .reduceXTicks(true)   // If 'false', every single x-axis tick label will be rendered.
      .rotateLabels(0)      // Angle to rotate x-axis labels.
      .showControls(true)   // Allow user to switch between 'Grouped' and 'Stacked' mode.
      .groupSpacing(0.1)    // Distance between each group of bars.
      .color(function(d, i) { // set color. fancy
        return d['color'];
      });

    chart.yAxis
      .tickFormat(d3.format(',.0f'));

    d3.select(el)
      .datum(data)
      .call(chart);

    nv.utils.windowResize(chart.update);

    return chart;
  }
};

$(document).ready(function() {
  // generate data from pandas json into d3 format (done by copying the example)
  var chart1data = window.convertKeys(window.averageByYear);
  var chart2data = window.convertKeys(window.averageByWeekday, window.WEEKDAYS_IN_ORDER);

  nv.addGraph(window.genGraph(chart1data, "#chart1 svg"));
  nv.addGraph(window.genGraph(chart2data, "#chart2 svg"));
});

In [ ]:
%%jsdoc plotData

window.headwayVsRidership = {{ merged | json }};

/*
 * As it turns out, pandas returns a dictionary with 
 * the top-level keys as the column headers, and values
 * as the list of values under that column.
 */
window.convertPlotData = function(data){
  var convertedData = [];
  // pandas gives each column the same number of
  // objects, arbitrarily choose one to iterate over
  for(var i in data[Object.keys(data)[0]]){
    var convertedDatum = {};
    for(var key in data){
      convertedDatum[key] = data[key][i];
    }
    convertedData.push(convertedDatum);
  }
  return convertedData;
}

In [ ]:
%%ihtml 1050
<html>
    <head>
        <script src="https://code.jquery.com/jquery-3.1.0.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.6/d3.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/nvd3/1.7.0/nv.d3.min.js"></script>
        {{ nvd3 | jsdoc }}
        <link id='nvd3style' rel='stylesheet' type='text/css' href='https://cdn.rawgit.com/novus/nvd3/v1.8.1/build/nv.d3.css'/>
    </head>
    <body>
        <div id="chart1" style="height: 500px"><svg></svg></div>
        <div id="chart2" style="height: 500px"><svg></svg></div>
    </body>
</html>

## Summary

Aside from just generally being prettier than the equivalent matplotlib/seaborn graph, this bar chart is interactive - and who doesn't love interactive graphs. We've also answered a variety of the questions we set out to:

- Stacking the yearly graph shows us an upward trend in overall ridership, and side-by-side shows us that this trend holds true across all 3 lines.
- We can stack the 'riders by weekday' graph to see somewhere between 9-10,000 people ride on the average day. For a system with ~20 buses and 3 lines, this isn't bad at all.
- We can side-by-side the 'riders by weekday' graph to see that the purple line is the most popular. Within Baltimore's geography, this makes sense as that bus travels through some of the city's most populated areas.
- Each line receives its highest ridership on Friday and sees minor drops on the weekend - which may indicate the system is popular among both tourists and commuters, but primarily functions as a commuter bus.

And we determined this with just a few minutes of Googling and wrestling with D3! Notably, there's substantially less code if you _don't_ do this in IPython notebook.

A brief note on the data-binding: We used `datum` instead of `data` here. This is NVD3 magic, and is based on some predefined keys that are expected in each element of the data array (note the `x` and `y` conversion we did in `convertKeys`). 

## Integrate angular.js with D3 for a cool interactive bar chart

The last graph in our first D3 notebook was static. Here, we're going to now show off the interactive power of D3 by combining it with angular, which is another JavaScript library.  Check out [Baby Name Popularity](projects/babyname/name_popularity.html).  

On the HTML side, we have introduced funny new attributes (called directives) with names that start with `ng`, e.g. `ng-repeat`, `ng-model`, `ng-click`.  These bind the specific HTML elements with JavaScript code.  On the JavaScript side, almost everything happens inside the controller `function NamesCtrl($scope)`.  You can think of the object `$scope` as representing all the variables and methods associated with this controller.

Below are a few features that you can play with which automatically update the graph:

<table style="width:100%">
<col width="60%">
<col width="40%">
<tr><th>
    These elements in HTML augmented with angular directives
</th>
<th>
    Call these functions in JavaScript
</th></tr>

<tr><td>
```
<div id="nameCtrl" ng-controller="NamesCtrl">
    ...
</div>
```
</td><td>
Associates all the code in this element (and sub-elements) with `function NamesCtrl($scope)`
</td></tr>

<tr><td>
```
<form ng-submit="addName()">
  <input type="text" ng-model="newName"
         placeholder="name to add">
  <input class="btn-primary" type="submit"
         value="add">
</form>```
</td><td>
This form calls `$scope.addName()` which adds values to `$scope.names`
</td></tr>

<tr><td>
`<input class="btn-primary" type="submit"
       ng-click="clearNames()" value="clear">`
</td><td>
Clears the names by calling `$scope.clearNames()`
</td></tr>

<tr><td>
`<ul class="unstyled">
  <li ng-repeat="name in names">
      ...
  </li>
</ul>`
</td><td>
The `ng-repeat="name in names"` directive creates a repeated list of `<li></li>` html elements which are each bound to `name`
</td></tr>

<tr><td>
`<input type="checkbox"
       ng-change="render()"
       ng-model="name.show">`
</td><td>
When the checkbox is clicked, the render function is called: `$scope.render()`.
This binds the checkbox to the model `name.show`
</td></tr>

<tr><td>
`<span>{{ name.name }}</span>`
</td><td>
The name in the span is bound to `name.name`
</td></tr>

</table>

The above code is all `angular code` that binds the user interface specifying which names to display with the JavaScript object `$scope.name`.

Our render function `$scope.render` is quite similar to that of the previous example except that it is inside the Controller (to get access to `$scope.names`) and has to filter for the names that we actually want to display data for, which is saved in `filteredData`:

```javascript
var filteredData = _.filter(scope.myData, function(d) {
    var activeNames = _.chain($scope.names)
            .filter(function(n) { return n.show; })
            .map(function(n) { return n.name; })
            .value();
    return _.contains(activeNames, d.label);
})
```


For more information, read a bit about [angular](https://angularjs.org), or more specifically look at this example of a [Todo List](https://angularjs.org/#add-some-control), which is what this example is patterned off of.

## On the web - live-query a backend

One of the more powerful aspects of web-based/interactive visualizations is the ability to query and render data in realtime. This allows us to, in some contexts, live-query for some subset of the data and effectively downsample this way.

- On the web, we always have to think about server -> client bandwidth considerations. 
- Remember, end-users don't necessarily have modern, powerful computers.
- Always assume the worst (the visualization equivalent of the UI/UX adage "assume the user is drunk")



## Do it in Python: the Bokeh Server
One of Bokeh's most powerful abstractions is the Bokeh Server, which lets you run a visualization server in the background that can live-query, process, filter, and compute visualizations using your more powerful backend infrastructure. For example, you might wish to let a user live-query site log data, or hit information, over varying timespans.

Check out the [Bokeh Gallery](http://bokeh.pydata.org/en/latest/docs/gallery.html#gallery) for some inspiration, and the [Bokeh Docs](http://bokeh.pydata.org/en/0.11.0/docs/user_guide/server.html) explain how the update functionality/synchronization works.

Under the hood, Bokeh is doing the same AJAX magic as the D3 example below, but you don't have to concern yourself as much with the details.

If you want to play around with the Bokeh server on Heroku, it does work! [With some fiddling](https://github.com/birdsarah/bokeh-server-heroku).

## Do it on the Web: D3.js and AJAX calls.

AJAX (advanced Javascript and XML) is a powerful web paradigm that allows for asynchronous calls between a backend server and a frontend browser. 

For example, if you're running code in Jupyter notebooks, you're using AJAX to 

1. send your code to the backend Jupyter server.
2. receive the output after the Jupyter server executes your code.

In the context of visualization, this allows us to live-query some subset of view of the data, process appropriately on the backend, and simply render on the frontend. Here's an example for the bus heatmap miniproject:

### JS (frontend)
```javascript
//NOTE: requires the underscore.js library
//NOTE: this code will slam the backend with requests because it has no delay.
// It's simply meant to showcase some asynchronous code that loops over small intervals.
// We could use the setTimeout function in javascript to delay this code.

_.range(24).foreach(function(hours){
    _.range(0, 60, 15).foreach(function(minutes){
        // use AJAX to query bus positions at 
        var swipes = d3.csv("/positions_at_time.csv?hours=" + hours + '&minutes=' + minutes)
        .get(function(error, rows){
          var points = rows.map(function(r){
            return  [
              +r.latitude,
              +r.longitude,
            ];
        });
    });
});
```

### Python (backend)
```python
# NOTE: code here written for Flask framework
@app.route('/positions_at_time.csv')
def positions_at_time():
  hours = int(request.args.get('hours', 12))
  minutes = int(request.args.get('minutes', 30))
  
  #could easily make this parameter modifiable
  date_for_data = datetime.date(2015, 9, 17)
  
  time_for_data = datetime.time(hours, minutes)
  
  datetime_for_data = datetime.datetime.combine(date_for_data, time_for_data)
  
  #2 minute window
  lower_bound = datetime_for_data - datetime.timedelta(seconds=60)
  upper_bound = datetime_for_data + datetime.timedelta(seconds=60)

  #JSON would work too, but CSV is actually better for D3
  return archive[lower_bound:upper_bound].to_csv()


```

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*